In [ ]:
!pip install scikit-fuzzy

In [167]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl

### Entradas e Saídas Fuzzy

Universo das entradas

In [ ]:
T = ctrl.Antecedent(np.arange(800, 1201, 1), 'temperatura')
P = ctrl.Antecedent(np.arange(4, 12.1, .1), 'pressao')

Funçoes de pertinência das variáveis de entrada

In [117]:
# temperatura
T['baixa'] = fuzz.trapmf(T.universe, [800, 800, 900,1000]) #FP trapezoidal
T['media'] = fuzz.trimf(T.universe,  [900,1000,1100]) #FP tringular
T['alta'] = fuzz.trapmf(T.universe,  [1000,1100,1200,1200]) #FP trapezoidal
#T.view()

#pressão
P['baixa'] = fuzz.gaussmf(P.universe, 4, 2) #FP gaussiana
P['media'] = fuzz.gaussmf(P.universe, 8, 1.3) #FP gaussiana
P['alta'] = fuzz.gaussmf(P.universe, 12, 2) #FP gaussiana
#P.view()

Domínio da variável de saída

In [119]:
V = ctrl.Consequent(np.arange(0, 3.1, .1), 'vazao')

Funçoes de pertinência da variável de saída

In [121]:
# vazão
V['baixa'] = fuzz.trapmf(V.universe,      [0, 0, 0.5, 1]) #FP trapezoidal
V['media-baixa'] = fuzz.trimf(V.universe, [0.5, 1, 1.5]) #FP tringular
V['media'] = fuzz.trimf(V.universe,       [1, 1.5, 2]) #FP tringular
V['media-alta'] = fuzz.trimf(V.universe,  [1.5, 2, 2.5]) #FP tringular
V['alta'] = fuzz.trapmf(V.universe,       [2, 2.5, 3, 3]) #FP trapezoidal
#V.view()

In [282]:
x = np.arange(0, 3.1, .1)
mfx = fuzz.trapmf(x, [0, 0, 0.5, 1]) #FP trapezoidal

# Defuzzify this membership function five ways
defuzz_centroid = fuzz.defuzz(x, mfx, 'centroid')  # Same as skfuzzy.centroid

### Regras Fuzzy

In [9]:
# Regra 1 - Se Temperatura é Baixa e Pressão é Baixa então Vazão é Baixa
R1 = ctrl.Rule(T['baixa'] & P['baixa'], V['baixa'])
#R1.view()

# Regra 2: Se Temperatura é Baixa e Pressão é Média então Vazão é Média- Baixa
R2 = ctrl.Rule(T['baixa'] & P['media'], V['media-baixa'])
#R2.view()

# Regra 3: Se Temperatura é Baixa e Pressão é Alta então Vazão é Média
R3 = ctrl.Rule(T['baixa'] & P['alta'], V['media'])
#R3.view()

# Regra 4: Se Temperatura é Média e Pressão é Baixa então Vazão é Média-Baixa
R4 = ctrl.Rule(T['media'] & P['baixa'], V['media-baixa'])
#R4.view()

# Regra 5: Se Temperatura é Média e Pressão é Média então Vazão é Média
R5 = ctrl.Rule(T['media'] & P['media'], V['media'])
#R5.view()

# Regra 6: Se Temperatura é Média e Pressão é Alta então Vazão é Média-Alta
R6 = ctrl.Rule(T['baixa'] & P['alta'], V['media-alta'])
#R6.view()

# Regra 7: Se Temperatura é Alta e Pressão é Baixa então Vazão é Média
R7= ctrl.Rule(T['alta'] & P['baixa'], V['media'])
#R7.view()

# Regra 8: Se Temperatura é Alta e Pressão é Média então Vazão é Média-Alta
R8= ctrl.Rule(T['alta'] & P['media'], V['media-alta'])
#R8.view()

# Regra 9: Se Temperatura é Alta e Pressão é Alta então Vazão é Alta
R9 = ctrl.Rule(T['alta'] & P['alta'], V['alta'])
#R9.view()

### Processo de Inferência Fuzzy

Dicionário de testes

In [126]:
dic = {}
dic['temperatura'] = [850,900,1000,1100,1150,880,950,1049,1089,1170]
dic['pressao'] = [8,4.5,9,6,10,11,7,5,7,9]
dic['vazao_desejada'] = [1.01,0.42,1.63,1.70,2.43,1.47,1.01,1.26,1.74,2.15]

Controlador Fuzzy

In [278]:
vazao_fuzzy = []
caldeira_crtl = ctrl.ControlSystem(rules = [R1, R2, R3, R4, R5, R6, R7, R8, R9])
engine = ctrl.ControlSystemSimulation(caldeira_crtl)

def predicao_fuzzy (t,p):

    engine.input['temperatura'] = t #entrada de temperatura
    engine.input['pressao'] = p #entrada de pressão
    engine.compute() #saída do controlador fuzzy
    vazao_fuzzy.append(round(engine.output['vazao'],3)) # lista dos valores crisp
    #V.view(sim=engine)

for i in range(len(dic['temperatura'])):
    predicao_fuzzy(dic['temperatura'][i],dic['pressao'][i])

### Cálculo do Erro

In [284]:
VF = np.array(vazao_fuzzy) # Vazão Fuzzy
VD = dic['vazao_desejada'] # Vazão Desejada
ER = abs(VF-VD)/VD *100 # Erro Relativo VD x VF

In [285]:
data = {'Vazao Desejada': dic['vazao_desejada'],
        'Vazao Estimada Fuzzy': vazao_fuzzy,
        'Erro Relativo %': ER}
df = pd.DataFrame(data)
df

,Vazao Desejada,Vazao Estimada Fuzzy,Erro Relativo %
0,1.01,1.109,9.801980
1,0.42,0.410,2.380952
2,1.63,1.465,10.122699
3,1.70,1.691,0.529412
4,2.43,2.389,1.687243
5,1.47,1.708,16.190476
6,1.01,1.061,5.049505
7,1.26,1.303,3.412698
8,1.74,1.774,1.954023
9,2.15,2.172,1.023256


Erro Relativo Médio

In [272]:
ERM = round(np.mean(df['Erro Relativo %']),3)
ERM

2.271